# Mamba vs. Transformer-based RALMs

The purpose of this project is to compare the performance of Retrieval Augmented Language Models (RALMs) based on the newly released Mamba architecture to those based on the more prevalent Transformer architecture. We will compare the [Mamba-Chat](https://huggingface.co/havenhq/mamba-chat) model to the [Dolly](https://huggingface.co/databricks/dolly-v2-3b) model. Both these models are approximately 2.8B parameters.

## Part 0: Imports and Environment Setup

First, to facilitate the use of this notebook in colab, load the rest of the repo. ONLY RUN THIS CODE IN COLAB.

In [ ]:
!wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
import colab_github
colab_github.github_auth(persistent_key=True)

In [ ]:
#after updating ssh key on github account if necessary, run below
!git clone git@github.com:abarton51/CS_4650_Project.git
!mv CS_4650_Project/* .

Next, install the required dependencies.

In [ ]:
!pip install datasets
!pip install transformers
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm
!pip install langchain

In [4]:
import sys
sys.path.append("src/")
import json
from datasets import load_dataset
from mamba_ralm import MambaRALM

# Part 1: Construct Vector Database

Now let us construct a Vector Database

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
import vector_store

In [ ]:
data_directory_path = "/content/drive/MyDrive/Mamba_RAG/CS_4650_Project/data/triviaqa-rc/evidence"
vector_store = vector_store.RAGVectorStore(data_directory_path, HuggingFaceBgeEmbeddings(), "FAISS")
vector_store.create_loader()
vector_store.load_docs()
vector_store.create_text_splitter(chunk_size=1000, chunk_overlap=0)
vector_store.split_text()
db = vector_store.get_db()

In [ ]:
query = "What is the capital of Japan?"
docs = db.similarity_search(query)
print(docs[0].page_content)
"""Example Output: Japan[a] is an island country in East Asia. It is in the northwest Pacific Ocean and is bordered on the west by the Sea of Japan, extending from the 
Sea of Okhotsk in the north toward the East China Sea, Philippine Sea, and Taiwan in the south. Japan is a part of the Ring of Fire, and spans an 
archipelago of 14,125 islands, with the five main islands being Hokkaido, Honshu (the "mainland"), Shikoku, Kyushu, and Okinawa. Tokyo is the country's 
capital and largest city, followed by Yokohama, Osaka, Nagoya, Sapporo, Fukuoka, Kobe, and Kyoto.
"""

# Part 2: Initialize Model

Initialize the model. Select the desired model here.

In [ ]:
model = MambaRALM()
model.predict("What is the capital of Japan?")

## Part 3: Perform Evaluation

Here we load the evaluation dataset, which we will use to evaluate our RAG system.

In [ ]:
evaluation_dataset_name = "wikipedia-dev"
evaluation_ds_filepath = "data/triviaqa-rc/qa/{ds_name}.json".format(ds_name=evaluation_dataset_name)

with open (evaluation_ds_filepath, "r") as json_file:

    eval_ds = json.load(json_file)["Data"]

MAX_EVALS = 100

evals = 0
accurate_evals = 0

#evaluation loop
for i in range(min(len(eval_ds), MAX_EVALS)):

    correct_answers = [answer.lower() for answer in eval_ds[i]["Answer"]["Aliases"]]

    model_answer = model.predict(eval_ds[i]["Question"]).lower()

    if model_answer in correct_answers:

        accurate_evals += 1

    evals += 1

print("Evaluation Complete")
print("Over {evals} datapoints, the model accurately answered {accurate_evals}, reflecting an over all accuracy of {overall_accuracy}".format(
    evals=evals,
    accurate_evals=accurate_evals,
    overall_accuracy=(accurate_evals/evals)
))
    
    